### 기상청 날씨 데이터
* BeautifulSoup의 find(), find_all() 함수을 사용해서 tag 선택하기
* 파싱한 데이터를 List에 Dict에 저장하기

In [11]:
import requests
from bs4 import BeautifulSoup

url = 'http://www.weather.go.kr/weather/forecast/mid-term-rss3.jsp'
res = requests.get(url)
print(res.status_code)
html = res.text
soup = BeautifulSoup(html, 'html.parser')

# <title> 태그 선택
print(type(soup.find('title')), soup.find('title').string)
print(type(soup.find_all('title')))
for title_tag in soup.find_all('title'):
    print(type(title_tag), title_tag.string)

200
<class 'bs4.element.Tag'> 기상청 육상 중기예보
<class 'bs4.element.ResultSet'>
<class 'bs4.element.Tag'> 기상청 육상 중기예보
<class 'bs4.element.Tag'> 전국 육상 중기예보 - 2021년 04월 28일 (수)요일 06:00 발표
<class 'bs4.element.Tag'> 전국 육상중기예보


In [19]:
print(len(soup.find_all('location', attrs={"wl_ver":"3"})))
location_first = soup.find('location', attrs={"wl_ver":"3"}) 
province = location_first.find('province').text
city = location_first.find('city').text
print(province, city)
for data in location_first.find_all('data'):
    mode = data.find('mode').text
    tmef = data.find('tmef').text
    wf = data.find('wf').text
    tmn = data.find('tmn').text
    tmx = data.find('tmx').text
    print(mode, tmn, tmx)

41
서울ㆍ인천ㆍ경기도 서울
A02 9 18
A02 9 18
A02 9 20
A02 9 20
A02 10 23
A02 10 23
A02 14 23
A02 14 23
A02 14 22
A02 14 22
A01 12 22
A01 12 22
A01 12 23


### 전국의 날씨정보를 추출
* location tag을 dict {} 에 저장
* 41개의 dict 을  list [] 에 저장
* json file 로 저장하기

In [28]:
import requests
from bs4 import BeautifulSoup

url = 'http://www.weather.go.kr/weather/forecast/mid-term-rss3.jsp'
res = requests.get(url)
print(res.status_code)
html = res.text
soup = BeautifulSoup(html, 'html.parser')

# 41개의 location 정보를 저장할 list 선언
location_list = list() # []
location_tags = soup.find_all('location', attrs={"wl_ver":"3"})
#print(len(location_tags))

for location in location_tags:
    # 1개의 location 태그를 저장할 dict 선언
    location_dict = dict() # {}
    location_dict['province'] = location.find('province').text
    location_dict['city'] = location.find('city').text
    data_tags = location.find_all('data')
    # 13개의 data 태그를 저장할 list 선언
    data_list = []
    for data in data_tags:
        # 1개의 data 태그를 저장할 dict 선언
        data_dict = {}
        data_dict['mode'] = data.find('mode').text
        data_dict['tmef'] = data.find('tmef').text
        data_dict['wf'] = data.find('wf').text
        data_dict['tmn'] = data.find('tmn').text
        data_dict['tmx'] = data.find('tmx').text
        data_list.append(data_dict)
        
    location_dict['datas'] = data_list   
    location_list.append(location_dict)
    

200


In [31]:
location_list[0:2]

[{'province': '서울ㆍ인천ㆍ경기도',
  'city': '서울',
  'datas': [{'mode': 'A02',
    'tmef': '2021-05-01 00:00',
    'wf': '구름많음',
    'tmn': '9',
    'tmx': '18'},
   {'mode': 'A02',
    'tmef': '2021-05-01 12:00',
    'wf': '맑음',
    'tmn': '9',
    'tmx': '18'},
   {'mode': 'A02',
    'tmef': '2021-05-02 00:00',
    'wf': '맑음',
    'tmn': '9',
    'tmx': '20'},
   {'mode': 'A02',
    'tmef': '2021-05-02 12:00',
    'wf': '맑음',
    'tmn': '9',
    'tmx': '20'},
   {'mode': 'A02',
    'tmef': '2021-05-03 00:00',
    'wf': '맑음',
    'tmn': '10',
    'tmx': '23'},
   {'mode': 'A02',
    'tmef': '2021-05-03 12:00',
    'wf': '구름많음',
    'tmn': '10',
    'tmx': '23'},
   {'mode': 'A02',
    'tmef': '2021-05-04 00:00',
    'wf': '흐림',
    'tmn': '14',
    'tmx': '23'},
   {'mode': 'A02',
    'tmef': '2021-05-04 12:00',
    'wf': '흐리고 비',
    'tmn': '14',
    'tmx': '23'},
   {'mode': 'A02',
    'tmef': '2021-05-05 00:00',
    'wf': '구름많고 비',
    'tmn': '14',
    'tmx': '22'},
   {'mode': 'A02',
    

In [32]:
import json

# location_list를 weather.json 파일로 저장하기
with open('data/weather.json','w') as file:
    json.dump(location_list, file)
    

In [34]:
import json
# weather.json 파일 읽기
with open('data/weather.json','r') as file:
    content = file.read()
    weather_json = json.loads(content)

weather_json    

[{'province': '서울ㆍ인천ㆍ경기도',
  'city': '서울',
  'datas': [{'mode': 'A02',
    'tmef': '2021-05-01 00:00',
    'wf': '구름많음',
    'tmn': '9',
    'tmx': '18'},
   {'mode': 'A02',
    'tmef': '2021-05-01 12:00',
    'wf': '맑음',
    'tmn': '9',
    'tmx': '18'},
   {'mode': 'A02',
    'tmef': '2021-05-02 00:00',
    'wf': '맑음',
    'tmn': '9',
    'tmx': '20'},
   {'mode': 'A02',
    'tmef': '2021-05-02 12:00',
    'wf': '맑음',
    'tmn': '9',
    'tmx': '20'},
   {'mode': 'A02',
    'tmef': '2021-05-03 00:00',
    'wf': '맑음',
    'tmn': '10',
    'tmx': '23'},
   {'mode': 'A02',
    'tmef': '2021-05-03 12:00',
    'wf': '구름많음',
    'tmn': '10',
    'tmx': '23'},
   {'mode': 'A02',
    'tmef': '2021-05-04 00:00',
    'wf': '흐림',
    'tmn': '14',
    'tmx': '23'},
   {'mode': 'A02',
    'tmef': '2021-05-04 12:00',
    'wf': '흐리고 비',
    'tmn': '14',
    'tmx': '23'},
   {'mode': 'A02',
    'tmef': '2021-05-05 00:00',
    'wf': '구름많고 비',
    'tmn': '14',
    'tmx': '22'},
   {'mode': 'A02',
    